<a href="https://colab.research.google.com/github/KatsiarynaDzibrova/taxi_trips_2/blob/main/taxi_trips_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn

In [ ]:
!pip install catboost
import catboost
from catboost import CatBoostRegressor, cv, Pool

In [ ]:
taxi_trips = pd.read_csv('/content/drive/My Drive/Taxi_Trips_-_2020.csv', parse_dates=['Trip Start Timestamp', 'Trip End Timestamp'])

In [ ]:
taxi_trips= taxi_trips.rename({'Trip ID': 'trip_id', 'Taxi ID': 'taxi_id',
                               'Trip Start Timestamp': 'trip_start_timestamp',
                              'Trip End Timestamp': 'trip_end_timestamp',
                              'Trip Seconds': 'trip_seconds', 'Trip Miles': 'trip_miles', 
                              'Pickup Census Tract': 'pickup_census_tract',
                              'Dropoff Census Tract': 'dropoff_census_tract', 'Fare': 'fare',
                              'Tips': 'tips', 'Tolls': 'tolls', 'Extras': 'extras', 'Trip Total': 'trip_total',
                              'Pickup Community Area' : 'pickup_community_area',
                              'Dropoff Community Area' : 'dropoff_community_area',
                              'Payment Type': 'payment_type', 'Company': 'company', 
                              'Pickup Centroid Latitude': 'pickup_centroid_latitude',
                              'Pickup Centroid Longitude': 'pickup_centroid_longitude',
                              'Pickup Centroid Location': 'pickup_centroid_location',
                              'Dropoff Centroid Latitude': 'dropoff_centroid_latitude',
                              'Dropoff Centroid Longitude':'dropoff_centroid_longitude',
                              'Dropoff Centroid  Location': 'dropoff_centroid_location'}, axis=1)

In [ ]:
taxi_trips = taxi_trips.drop(columns=['pickup_census_tract', 'dropoff_census_tract', 'trip_id', 'taxi_id', 'trip_end_timestamp', 'tips', 'tolls', 'extras', 'trip_total', 'payment_type', 'fare', 'pickup_centroid_location', 'dropoff_centroid_location'])

In [ ]:
taxi_trips = taxi_trips.dropna(subset=['trip_seconds'])

In [ ]:
taxi_trips

,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,2020-01-01,240.0,0.60,8.0,8.0,Choice Taxi Association,41.899602,-87.633308,41.899602,-87.633308
1,2020-01-01,600.0,1.00,8.0,8.0,Star North Management LLC,41.892042,-87.631864,41.895033,-87.619711
2,2020-01-01,383.0,0.69,8.0,8.0,Flash Cab,41.895033,-87.619711,41.898332,-87.620763
3,2020-01-01,1861.0,3.16,8.0,32.0,City Service,41.899156,-87.626211,41.877406,-87.621972
4,2020-01-01,960.0,11.10,76.0,21.0,Star North Management LLC,41.980264,-87.913625,41.938666,-87.711211
...,...,...,...,...,...,...,...,...,...,...
3597302,2020-11-01,660.0,0.30,76.0,NaN,Blue Ribbon Taxi Association Inc.,41.980264,-87.913625,NaN,NaN
3597303,2020-11-01,373.0,2.61,6.0,3.0,Flash Cab,41.944227,-87.655998,41.965812,-87.655879
3597304,2020-11-01,2654.0,7.38,8.0,38.0,Flash Cab,41.899602,-87.633308,41.812949,-87.617860
3597305,2020-11-01,300.0,0.50,8.0,8.0,Taxi Affiliation Services,41.899602,-87.633308,41.899602,-87.633308


## 1. CatBoost

1.1. Without standartization

In [ ]:
taxi_trips_mean = taxi_trips.copy()

In [ ]:
for col in taxi_trips_mean.columns:
  print(col)
  if col == 'company': 
    to_fill = ''
    print('comp')
  elif  col in ['pickup_community_area','dropoff_community_area', 'pickup_census_tract', 'dropoff_census_tract']:
    to_fill = 0
  else: to_fill = taxi_trips_mean[col].mean()
  print(to_fill, taxi_trips_mean[col].isna().count())
  taxi_trips_mean[col] = taxi_trips_mean[col].fillna(to_fill)

trip_start_timestamp
2020-03-22 09:22:47.954599936 3595944
trip_seconds
857.8193456294091 3595944
trip_miles
3.5871026881545762 3595944
pickup_community_area
0 3595944
dropoff_community_area
0 3595944
company
comp
 3595944
pickup_centroid_latitude
41.896338412863585 3595944
pickup_centroid_longitude
-87.66575581234449 3595944
dropoff_centroid_latitude
41.89468724043545 3595944
dropoff_centroid_longitude
-87.65267616351382 3595944


In [ ]:
taxi_trips_mean.pickup_community_area = taxi_trips_mean.pickup_community_area.map(int)
taxi_trips_mean.dropoff_community_area = taxi_trips_mean.dropoff_community_area.map(int)

In [ ]:
taxi_trips_mean['hour'] = taxi_trips_mean.trip_start_timestamp.dt.hour
taxi_trips_mean['weekday'] = taxi_trips_mean.trip_start_timestamp.dt.weekday
taxi_trips_mean['dayofyear'] = taxi_trips.trip_start_timestamp.dt.dayofyear

In [ ]:
y = taxi_trips_mean.trip_seconds
X = taxi_trips_mean.drop(['trip_seconds'],axis=1)

In [ ]:
def learn(X_train, X_test, y_train, y_test):
  categorical_features_indices = np.where((X_train.dtypes == np.bool) | (X_train.dtypes == np.object))[0]
  for i in range(len(X_train.columns)):
    if (X_train.columns[i] in ['pickup_community_area','dropoff_community_area', 'pickup_census_tract', 'dropoff_census_tract']):
      categorical_features_indices = np.append(categorical_features_indices, i)
  print(categorical_features_indices)
  model = CatBoostRegressor(
        learning_rate=0.01,
        depth=11,
        l2_leaf_reg=43,
        iterations=100,
        min_data_in_leaf=12,
        per_float_feature_quantization=['0:border_count=1024'],
        random_strength=16,
        od_wait=20,
        random_seed=42,
        verbose=20,
    )
  model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_test, y_test))
  pred = model.predict(X_test)
  print(model.get_feature_importance())
  return sklearn.metrics.mean_absolute_error(y_test, pred)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
error_third = learn(X_train, X_test, y_train, y_test)

[4 2 3]
0:	learn: 1628.1335334	test: 1676.9328396	best: 1676.9328396 (0)	total: 1.49s	remaining: 2m 27s
20:	learn: 1563.1805878	test: 1618.5902208	best: 1618.5902208 (20)	total: 27.1s	remaining: 1m 41s
40:	learn: 1542.4255355	test: 1603.7818113	best: 1603.7818113 (40)	total: 49.6s	remaining: 1m 11s
60:	learn: 1522.4501183	test: 1592.8852109	best: 1592.8852109 (60)	total: 1m 14s	remaining: 47.7s
80:	learn: 1512.4612733	test: 1587.2684535	best: 1587.2684535 (80)	total: 1m 37s	remaining: 22.8s
99:	learn: 1504.7262384	test: 1583.4515709	best: 1583.4335836 (98)	total: 1m 59s	remaining: 0us

bestTest = 1583.433584
bestIteration = 98

Shrink model to first 99 iterations.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
error_tenth = learn(X_train, X_test, y_train, y_test)

[4 2 3]
0:	learn: 1638.7321819	test: 1668.2188579	best: 1668.2188579 (0)	total: 1.92s	remaining: 3m 10s
20:	learn: 1573.1201458	test: 1605.7449353	best: 1605.7449353 (20)	total: 31.9s	remaining: 1m 59s
40:	learn: 1548.8089621	test: 1585.8347277	best: 1585.8347277 (40)	total: 1m 4s	remaining: 1m 33s
60:	learn: 1529.1199726	test: 1574.2297337	best: 1574.2297337 (60)	total: 1m 39s	remaining: 1m 3s
80:	learn: 1514.7389914	test: 1567.9024282	best: 1567.9024282 (80)	total: 2m 15s	remaining: 31.7s
99:	learn: 1506.5879238	test: 1565.8100268	best: 1565.6288098 (98)	total: 2m 48s	remaining: 0us

bestTest = 1565.62881
bestIteration = 98

Shrink model to first 99 iterations.


In [ ]:
error_third

305.997996010371

In [ ]:
error_tenth

301.53415080621374

### 2.1. With standartization

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for col in X.columns:
  if  col in ['company', 'pickup_community_area','dropoff_community_area', 'pickup_census_tract', 'dropoff_census_tract', 'trip_start_timestamp']:
    continue
  x = pd.DataFrame(X[col])
  scaler = preprocessing.StandardScaler()
  x_scaled = scaler.fit_transform(x)
  X[col] = x_scaled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
error_tenth = learn(X_train, X_test, y_train, y_test)
error_tenth

[4 2 3]
0:	learn: 1711.6464913	test: 1739.9478078	best: 1739.9478078 (0)	total: 1.82s	remaining: 3m
20:	learn: 1679.8810952	test: 1708.7826795	best: 1708.7826795 (20)	total: 37.3s	remaining: 2m 20s
40:	learn: 1657.3797999	test: 1686.7722235	best: 1686.7722235 (40)	total: 1m 12s	remaining: 1m 44s
60:	learn: 1641.0860159	test: 1670.8758524	best: 1670.8758524 (60)	total: 1m 47s	remaining: 1m 8s
80:	learn: 1629.3298990	test: 1659.3588214	best: 1659.3588214 (80)	total: 2m 23s	remaining: 33.7s
99:	learn: 1621.0959505	test: 1651.3744683	best: 1651.3744683 (99)	total: 2m 56s	remaining: 0us

bestTest = 1651.374468
bestIteration = 99

[ 1.08763526 52.71109436  0.          0.          0.          7.78823472
 13.62381057  8.19206058 10.52023962  3.99478509  1.03370761  1.04843218]


393.3286144516205